In [1]:

import time
import numpy as np
import cv2
import scipy.special
import matplotlib.pyplot as plt
import os
import csv

from os import listdir
from os.path import isfile, join
import pandas
from scipy import ndimage
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def detect_orange(img):
  img = cv2.resize(img,(184,409))

  hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  hsv = cv2.blur(cv2.cvtColor(img,cv2.COLOR_BGR2HSV),(10,10))

  lower_orange = np.array([10,120,120])
  upper_orange = np.array([20,255,255])
  mask1 = cv2.inRange(hsv, lower_orange, upper_orange)
  #cv2.imshow('mask1',mask1)
  white_val = 0
  for row in range(len(mask1)):
      white_val += sum(mask1[row])/255
  return white_val

In [4]:
def to_crop_img(image):
  #image = cv2.imread(img_name)


  img = image.copy()
  image = cv2.resize(img,(len(img[0])//5,len(img)//5))

  img = cv2.blur(img,(15,15))

  img = cv2.resize(img,(len(img[0])//5,len(img)//5))

  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Find Canny edges
  #edged = cv2.Canny(img, 25, 250)
  edged = cv2.threshold(img, 125, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


  contours, hierarchy = cv2.findContours(edged, 
      cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  max_position = 0
  max_value = 0
  for i in range(len(contours)):
      #print(contours[i].shape)
      if(contours[i].shape[0]>max_value):
          max_value = contours[i].shape[0]
          max_position = i
          

  x,y,w,h = cv2.boundingRect(contours[max_position])
  #print(w,' ',h,' ',w/h)
  crop_img = image[y:y+h, x:x+w]

  if(w/h > 0.59):
      rotate = ndimage.rotate(crop_img, 0-((w/h-0.54)*100),cval=205.0)
      img_rotate = cv2.blur(rotate,(1,1))
      img_rotate = cv2.cvtColor(img_rotate, cv2.COLOR_BGR2GRAY)
      rotate_edged = cv2.threshold(img_rotate, 125, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
      #cv2.imshow('img_rotate',rotate_edged)
      contours_rotate, hierarchy_rotate = cv2.findContours(rotate_edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      max_position_rotate = 0
      max_value_rotate = 0
      for i in range(len(contours_rotate)):
          #print(contours[i].shape)
          if(contours_rotate[i].shape[0]>max_value_rotate):
              max_value_rotate = contours_rotate[i].shape[0]
              max_position_rotate = i
      x_rotate,y_rotate,w_rotate,h_rotate = cv2.boundingRect(contours_rotate[max_position_rotate])
      crop_img_rotate = rotate[y_rotate:y_rotate+h_rotate, x_rotate:x_rotate+w_rotate]
      #print(w_rotate/h_rotate)
      ratio_unclockwise = w_rotate/h_rotate
      if(w_rotate/h_rotate >0.59):
          rotate = ndimage.rotate(crop_img, ((w/h-0.57)*100),cval=205.0)
          img_rotate = cv2.blur(rotate,(1,1))
          img_rotate = cv2.cvtColor(img_rotate, cv2.COLOR_BGR2GRAY)
          rotate_edged = cv2.threshold(img_rotate, 125, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
          #cv2.imshow('img_rotate',rotate_edged)
          contours_rotate, hierarchy_rotate = cv2.findContours(rotate_edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
          max_position_rotate = 0
          max_value_rotate = 0
          for i in range(len(contours_rotate)):
              #print(contours[i].shape)
              if(contours_rotate[i].shape[0]>max_value_rotate):
                  max_value_rotate = contours_rotate[i].shape[0]
                  max_position_rotate = i
          x_rotate,y_rotate,w_rotate,h_rotate = cv2.boundingRect(contours_rotate[max_position_rotate])
          crop_img_rotate = rotate[y_rotate:y_rotate+h_rotate, x_rotate:x_rotate+w_rotate]
          #print(w_rotate/h_rotate)
          ratio_clockwise = w_rotate/h_rotate
          if (ratio_clockwise>ratio_unclockwise):
            rotate = ndimage.rotate(crop_img, 0-((w/h-0.54)*100),cval=205.0)
            img_rotate = cv2.blur(rotate,(1,1))
            img_rotate = cv2.cvtColor(img_rotate, cv2.COLOR_BGR2GRAY)
            rotate_edged = cv2.threshold(img_rotate, 125, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
            #cv2.imshow('img_rotate',rotate_edged)
            contours_rotate, hierarchy_rotate = cv2.findContours(rotate_edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            max_position_rotate = 0
            max_value_rotate = 0
            for i in range(len(contours_rotate)):
                #print(contours[i].shape)
                if(contours_rotate[i].shape[0]>max_value_rotate):
                    max_value_rotate = contours_rotate[i].shape[0]
                    max_position_rotate = i
            x_rotate,y_rotate,w_rotate,h_rotate = cv2.boundingRect(contours_rotate[max_position_rotate])
            crop_img_rotate = rotate[y_rotate:y_rotate+h_rotate, x_rotate:x_rotate+w_rotate]
            #print(w_rotate/h_rotate)
      #cv2.imshow('rotate',crop_img_rotate)
      crop_img = crop_img_rotate #
  return crop_img

In [5]:
def avg_in_scale(crop_img):
  crop_img = img = cv2.resize(crop_img,(275,512))
  avg_scale_from_crop = []
  count = 0
  for row_line in range(len(crop_img)//30-2):
    for col_line in range(len(crop_img[0])//30-2):
      img_scale = crop_img[(row_line+1)*30:(row_line+1)*30+30, (col_line+1)*30:(col_line+1)*30+30]
      avg_pix = 0
      for row in range(len(img_scale)):
        for col in range(len(img_scale[0])):
          avg_pix += sum(img_scale[row][col])/3
      avg_scale_from_crop.append(avg_pix)
      count+=1
  if (count<105):
    for i in range(105-count):
      avg_scale_from_crop.append(0)
  elif (count>105):
    for i in range(count-105):
      avg_scale_from_crop.pop()

  return np.array(avg_scale_from_crop)

In [6]:
def detect_yellow(img):
  img = cv2.resize(img,(184,409))

  hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  hsv = cv2.blur(cv2.cvtColor(img,cv2.COLOR_BGR2HSV),(10,10))

  lower_yellow = np.array([18,120,120])
  upper_yellow = np.array([25,255,255])
  mask1 = cv2.inRange(hsv, lower_yellow, upper_yellow)
  #cv2.imshow('mask1',mask1)
  white_val = 0
  for row in range(len(mask1)):
      white_val += sum(mask1[row])/255
  return white_val

In [7]:
def detect_green(img):
  img = cv2.resize(img,(184,409))

  hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  hsv = cv2.blur(cv2.cvtColor(img,cv2.COLOR_BGR2HSV),(10,10))

  lower_green = np.array([35,120,120])
  upper_green = np.array([55,255,255])
  mask1 = cv2.inRange(hsv, lower_green, upper_green)
  #cv2.imshow('mask1',mask1)
  white_val = 0
  for row in range(len(mask1)):
      white_val += sum(mask1[row])/255
  return white_val

In [8]:
# neural network class definition
class neuralNetwork :

    # initialise the neural network
    def __init__(self, inputnodes, outputnodes,learningrate,bias,send_weight=False,weight=0) :

        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        
        self.onodes = outputnodes

        #set weight
        if(send_weight):
          self.w = weight
        else:
          self.w = np.random.normal(0.0, pow(self.onodes, -0.5),(self.onodes, self.inodes))

        # learning rate
        self.lr = learningrate
        self.bias = bias

        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        #self.derivative_function = lambda x: 1 if x>0 else 0 if x==0 else -1

        self.dw = 0
        pass
   
    

    # train the neural network
    def train(self, inputs_list, targets_list) :
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        # print('targets = ', targets)

        
        w_dot_input = np.dot(self.w, inputs)
        #print(w_dot_input)
        
        outputs = self.activation_function(w_dot_input+1)
        #print('final_outputs = ', outputs)

        # output layer error is the (target actual)
        output_errors = targets-outputs
        #print('output_errors = ', output_errors)
        
        # update the weights 
        self.dw = self.lr * np.dot(output_errors,np.transpose(inputs))
        #print(np.multiply( output_errors,self.derivative_function(outputs) ))
        #print(output_errors*self.derivative_function(outputs))
        #self.dw = self.lr * np.dot(output_errors,np.transpose(inputs))#+(0.9*self.dw)
        #self.dw = self.lr * np.dot( np.multiply( output_errors,self.derivative_function(outputs) ) ,np.transpose(inputs))+(0.9*self.dw)
        self.dw = self.lr * np.dot((output_errors*outputs * (1.0-outputs)),np.transpose(inputs)) +(0.9*self.dw)
        self.w += self.dw
        self.bias +=  self.lr *sum(output_errors)
        


        # print("*********")
        pass

    # query the neural network
    def query(self, inputs_list) :

        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T


        output = np.dot(self.w, inputs)

        # calculate the signals emerging 
        final_outputs = self.activation_function(output)

        return final_outputs
    
   
    # def write_weight(self) :
    #     f = open('who.txt','w',encoding='utf-8')
    #     f.write(str(self.who))
    #     f.close()
    #     f = open('wih.txt','w',encoding='utf-8')
    #     f.write(str(self.wih))
    #     f.close()
    #     print('already write!! ')
        

In [9]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/train_data_use_color2.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/train_data_use_color2.zip
   creating: train_data/
  inflating: train_data/maeyupa.0.jpg  
  inflating: train_data/maeyupa.1.jpg  
  inflating: train_data/maeyupa.100.jpg  
  inflating: train_data/maeyupa.101.jpg  
  inflating: train_data/maeyupa.108.jpg  
  inflating: train_data/maeyupa.109.jpg  
  inflating: train_data/maeyupa.110.jpg  
  inflating: train_data/maeyupa.111.jpg  
  inflating: train_data/maeyupa.112.jpg  
  inflating: train_data/maeyupa.113.jpg  
  inflating: train_data/maeyupa.12.jpg  
  inflating: train_data/maeyupa.120.jpg  
  inflating: train_data/maeyupa.121.jpg  
  inflating: train_data/maeyupa.122.jpg  
  inflating: train_data/maeyupa.123.jpg  
  inflating: train_data/maeyupa.124.jpg  
  inflating: train_data/maeyupa.125.jpg  
  inflating: train_data/maeyupa.13.jpg  
  inflating: train_data/maeyupa.1

In [10]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_data_use_color2.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_data_use_color2.zip
   creating: test_data/
  inflating: test_data/maeyupa.10.jpg  
  inflating: test_data/maeyupa.102.jpg  
  inflating: test_data/maeyupa.103.jpg  
  inflating: test_data/maeyupa.104.jpg  
  inflating: test_data/maeyupa.105.jpg  
  inflating: test_data/maeyupa.106.jpg  
  inflating: test_data/maeyupa.107.jpg  
  inflating: test_data/maeyupa.11.jpg  
  inflating: test_data/maeyupa.114.jpg  
  inflating: test_data/maeyupa.115.jpg  
  inflating: test_data/maeyupa.116.jpg  
  inflating: test_data/maeyupa.117.jpg  
  inflating: test_data/maeyupa.118.jpg  
  inflating: test_data/maeyupa.119.jpg  
  inflating: test_data/maeyupa.126.jpg  
  inflating: test_data/maeyupa.127.jpg  
  inflating: test_data/maeyupa.128.jpg  
  inflating: test_data/maeyupa.129.jpg  
  inflating: test_data/maeyupa.130.jpg  
  inflat

In [11]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_after_train.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_after_train.zip
   creating: test_after_train/
  inflating: test_after_train/maeyupa.0.jpg  
  inflating: test_after_train/maeyupa.1.jpg  
  inflating: test_after_train/maeyupa.10.jpg  
  inflating: test_after_train/maeyupa.11.jpg  
  inflating: test_after_train/maeyupa.12.jpg  
  inflating: test_after_train/maeyupa.13.jpg  
  inflating: test_after_train/maeyupa.14.jpg  
  inflating: test_after_train/maeyupa.15.jpg  
  inflating: test_after_train/maeyupa.16.jpg  
  inflating: test_after_train/maeyupa.17.jpg  
  inflating: test_after_train/maeyupa.18.jpg  
  inflating: test_after_train/maeyupa.19.jpg  
  inflating: test_after_train/maeyupa.2.jpg  
  inflating: test_after_train/maeyupa.20.jpg  
  inflating: test_after_train/maeyupa.21.jpg  
  inflating: test_after_train/maeyupa.22.jpg  
  inflating: test_after_train/mae

In [12]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_after_train2.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_after_train2.zip
   creating: test_after_train2/
  inflating: test_after_train2/maeyupa.0.jpg  
  inflating: test_after_train2/maeyupa.1.jpg  
  inflating: test_after_train2/maeyupa.10.jpg  
  inflating: test_after_train2/maeyupa.11.jpg  
  inflating: test_after_train2/maeyupa.12.jpg  
  inflating: test_after_train2/maeyupa.13.jpg  
  inflating: test_after_train2/maeyupa.14.jpg  
  inflating: test_after_train2/maeyupa.15.jpg  
  inflating: test_after_train2/maeyupa.16.jpg  
  inflating: test_after_train2/maeyupa.17.jpg  
  inflating: test_after_train2/maeyupa.18.jpg  
  inflating: test_after_train2/maeyupa.19.jpg  
  inflating: test_after_train2/maeyupa.2.jpg  
  inflating: test_after_train2/maeyupa.20.jpg  
  inflating: test_after_train2/maeyupa.21.jpg  
  inflating: test_after_train2/maeyupa.22.jpg  
  inflating: te

In [13]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_after_train3.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_after_train3.zip
   creating: test_after_train3/
  inflating: test_after_train3/maeyupa.0.jpg  
  inflating: test_after_train3/maeyupa.1.jpg  
  inflating: test_after_train3/maeyupa.10.jpg  
  inflating: test_after_train3/maeyupa.11.jpg  
  inflating: test_after_train3/maeyupa.12.jpg  
  inflating: test_after_train3/maeyupa.13.jpg  
  inflating: test_after_train3/maeyupa.14.jpg  
  inflating: test_after_train3/maeyupa.15.jpg  
  inflating: test_after_train3/maeyupa.16.jpg  
  inflating: test_after_train3/maeyupa.17.jpg  
  inflating: test_after_train3/maeyupa.18.jpg  
  inflating: test_after_train3/maeyupa.19.jpg  
  inflating: test_after_train3/maeyupa.2.jpg  
  inflating: test_after_train3/maeyupa.20.jpg  
  inflating: test_after_train3/maeyupa.21.jpg  
  inflating: test_after_train3/maeyupa.22.jpg  
  inflating: te

In [14]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_after_train4.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_after_train4.zip
   creating: test_after_train4/
  inflating: test_after_train4/maeyupa.0.jpg  
  inflating: test_after_train4/maeyupa.1.jpg  
  inflating: test_after_train4/maeyupa.10.jpg  
  inflating: test_after_train4/maeyupa.11.jpg  
  inflating: test_after_train4/maeyupa.12.jpg  
  inflating: test_after_train4/maeyupa.13.jpg  
  inflating: test_after_train4/maeyupa.14.jpg  
  inflating: test_after_train4/maeyupa.15.jpg  
  inflating: test_after_train4/maeyupa.16.jpg  
  inflating: test_after_train4/maeyupa.17.jpg  
  inflating: test_after_train4/maeyupa.18.jpg  
  inflating: test_after_train4/maeyupa.19.jpg  
  inflating: test_after_train4/maeyupa.2.jpg  
  inflating: test_after_train4/maeyupa.20.jpg  
  inflating: test_after_train4/maeyupa.21.jpg  
  inflating: test_after_train4/maeyupa.22.jpg  
  inflating: te

In [15]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_after_train5.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_after_train5.zip
   creating: test_after_train5/
  inflating: test_after_train5/maeyupa.0.jpg  
  inflating: test_after_train5/maeyupa.1.jpg  
  inflating: test_after_train5/maeyupa.10.jpg  
  inflating: test_after_train5/maeyupa.11.jpg  
  inflating: test_after_train5/maeyupa.12.jpg  
  inflating: test_after_train5/maeyupa.13.jpg  
  inflating: test_after_train5/maeyupa.14.jpg  
  inflating: test_after_train5/maeyupa.15.jpg  
  inflating: test_after_train5/maeyupa.16.jpg  
  inflating: test_after_train5/maeyupa.17.jpg  
  inflating: test_after_train5/maeyupa.18.jpg  
  inflating: test_after_train5/maeyupa.19.jpg  
  inflating: test_after_train5/maeyupa.2.jpg  
  inflating: test_after_train5/maeyupa.20.jpg  
  inflating: test_after_train5/maeyupa.21.jpg  
  inflating: test_after_train5/maeyupa.22.jpg  
  inflating: te

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_after_train6.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_after_train6.zip


In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_after_train7.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_after_train7.zip
   creating: test_after_train7/
  inflating: test_after_train7/maeyupa.0.jpg  
  inflating: test_after_train7/maeyupa.1.jpg  
  inflating: test_after_train7/maeyupa.10.jpg  
  inflating: test_after_train7/maeyupa.11.jpg  
  inflating: test_after_train7/maeyupa.12.jpg  
  inflating: test_after_train7/maeyupa.13.jpg  
  inflating: test_after_train7/maeyupa.14.jpg  
  inflating: test_after_train7/maeyupa.15.jpg  
  inflating: test_after_train7/maeyupa.16.jpg  
  inflating: test_after_train7/maeyupa.17.jpg  
  inflating: test_after_train7/maeyupa.18.jpg  
  inflating: test_after_train7/maeyupa.19.jpg  
  inflating: test_after_train7/maeyupa.2.jpg  
  inflating: test_after_train7/maeyupa.20.jpg  
  inflating: test_after_train7/maeyupa.21.jpg  
  inflating: test_after_train7/maeyupa.22.jpg  
  inflating: te

In [ ]:
mypath = 'train_data/'
imagePaths = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
testpath = 'test_data/'
imageTestPaths = [testpath+f for f in listdir(testpath) if isfile(join(testpath, f))]

In [ ]:
testAfterTrainpath = 'test_after_train/'
imageTestAfterTrainPaths = [testAfterTrainpath+f for f in listdir(testAfterTrainpath) if isfile(join(testAfterTrainpath, f))]

In [ ]:
testAfterTrainpath2 = 'test_after_train2/'
imageTestAfterTrainPaths2 = [testAfterTrainpath2+f for f in listdir(testAfterTrainpath2) if isfile(join(testAfterTrainpath2, f))]

In [ ]:
testAfterTrainpath3 = 'test_after_train3/'
imageTestAfterTrainPaths3 = [testAfterTrainpath3+f for f in listdir(testAfterTrainpath3) if isfile(join(testAfterTrainpath3, f))]

In [ ]:
testAfterTrainpath4 = 'test_after_train4/'
imageTestAfterTrainPaths4 = [testAfterTrainpath4+f for f in listdir(testAfterTrainpath4) if isfile(join(testAfterTrainpath4, f))]

In [ ]:
testAfterTrainpath5 = 'test_after_train5/'
imageTestAfterTrainPaths5 = [testAfterTrainpath5+f for f in listdir(testAfterTrainpath5) if isfile(join(testAfterTrainpath5, f))]

In [ ]:
testAfterTrainpath6 = 'test_after_train6/'
imageTestAfterTrainPaths6 = [testAfterTrainpath6+f for f in listdir(testAfterTrainpath6) if isfile(join(testAfterTrainpath6, f))]

In [ ]:
testAfterTrainpath7 = 'test_after_train7/'
imageTestAfterTrainPaths7 = [testAfterTrainpath7+f for f in listdir(testAfterTrainpath7) if isfile(join(testAfterTrainpath7, f))]

In [ ]:
# train_list = []
# train_list_ans = []
# test_list = []
# test_list_ans = []
# for (i, img_path) in enumerate(imagePaths):
#       img = cv2.imread(img_path)
#       orange_val = detect_orange(img)
#       yellow_val = detect_yellow(img)
#       green_val = detect_green(img)
#       color_list = np.array([orange_val,yellow_val,green_val])
#       #train_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01)
#       train_list.append(color_list)
#       product_name = img_path.split(".")[0].split("/")[1]
#       product_class = 0
#       if (product_name == "maeyupa"):
#           product_class = 1
#       elif (product_name == "ocean"):
#           product_class = 2
#       elif (product_name == "zapnuer"):
#           product_class = 3
#       elif (product_name == "zapnuerchick"):
#           product_class = 4
      
#       targets = np.zeros(4) + 0.01
      
      
#       targets[product_class-1] = 0.99
#       train_list_ans.append(targets)

# for (i, img_path_test) in enumerate(imageTestPaths):
#       img = cv2.imread(img_path_test)
#       orange_val = detect_orange(img)
#       yellow_val = detect_yellow(img)
#       green_val = detect_green(img)
#       color_list = np.array([orange_val,yellow_val,green_val])
#       #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
#       test_list.append(color_list)
#       product_name = img_path_test.split(".")[0].split("/")[1]
#       product_class = 0
#       if (product_name == "maeyupa"):
#           product_class = 1
#       elif (product_name == "ocean"):
#           product_class = 2
#       elif (product_name == "zapnuer"):
#           product_class = 3
#       elif (product_name == "zapnuerchick"):
#           product_class = 4

#       test_list_ans.append(product_class-1)
              
      
      

In [ ]:
train_list = []
train_list_ans = []
test_list = []
test_list_ans = []
for (i, img_path) in enumerate(imagePaths):
      img = cv2.imread(img_path)
      crop_img = to_crop_img(img) 
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #train_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01)
      train_list.append(avg_in_scale(crop_img))
      product_name = img_path.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4
      
      targets = np.zeros(4) + 0.01
      
      
      targets[product_class-1] = 0.99
      train_list_ans.append(targets)

for (i, img_path_test) in enumerate(imageTestPaths):
      img = cv2.imread(img_path_test)
      crop_img = to_crop_img(img)
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
      test_list.append(avg_in_scale(crop_img))
      product_name = img_path_test.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4

      test_list_ans.append(product_class-1)
              
      
      

In [ ]:
test_after_train_list = []
test_after_train_list_ans = []
for (i, img_path_test_after_train) in enumerate(imageTestAfterTrainPaths):
      img = cv2.imread(img_path_test_after_train)
      crop_img = to_crop_img(img)
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
      test_after_train_list.append(avg_in_scale(crop_img))
      product_name = img_path_test_after_train.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4

      test_after_train_list_ans.append(product_class-1)

In [ ]:
test_after_train_list2 = []
test_after_train_list_ans2 = []
for (i, img_path_test_after_train2) in enumerate(imageTestAfterTrainPaths2):
      img = cv2.imread(img_path_test_after_train2)
      crop_img = to_crop_img(img)
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
      test_after_train_list2.append(avg_in_scale(crop_img))
      product_name = img_path_test_after_train2.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4

      test_after_train_list_ans2.append(product_class-1)

In [ ]:
test_after_train_list3 = []
test_after_train_list_ans3 = []
for (i, img_path_test_after_train3) in enumerate(imageTestAfterTrainPaths3):
      img = cv2.imread(img_path_test_after_train3)
      crop_img = to_crop_img(img)
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
      test_after_train_list3.append(avg_in_scale(crop_img))
      product_name = img_path_test_after_train3.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4

      test_after_train_list_ans3.append(product_class-1)

In [ ]:
test_after_train_list4 = []
test_after_train_list_ans4 = []
for (i, img_path_test_after_train4) in enumerate(imageTestAfterTrainPaths4):
      img = cv2.imread(img_path_test_after_train4)
      crop_img = to_crop_img(img)
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
      test_after_train_list4.append(avg_in_scale(crop_img))
      product_name = img_path_test_after_train4.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4

      test_after_train_list_ans4.append(product_class-1)

In [ ]:
test_after_train_list5 = []
test_after_train_list_ans5 = []
for (i, img_path_test_after_train5) in enumerate(imageTestAfterTrainPaths5):
      img = cv2.imread(img_path_test_after_train5)
      crop_img = to_crop_img(img)
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
      test_after_train_list5.append(avg_in_scale(crop_img))
      product_name = img_path_test_after_train5.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4

      test_after_train_list_ans5.append(product_class-1)

In [ ]:
test_after_train_list6 = []
test_after_train_list_ans6 = []
for (i, img_path_test_after_train6) in enumerate(imageTestAfterTrainPaths6):
      img = cv2.imread(img_path_test_after_train6)
      crop_img = to_crop_img(img)
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
      test_after_train_list6.append(avg_in_scale(crop_img))
      product_name = img_path_test_after_train6.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4

      test_after_train_list_ans6.append(product_class-1)

In [ ]:
test_after_train_list7 = []
test_after_train_list_ans7 = []
for (i, img_path_test_after_train7) in enumerate(imageTestAfterTrainPaths7):
      img = cv2.imread(img_path_test_after_train7)
      crop_img = to_crop_img(img)
      # orange_val = detect_orange(img)
      # yellow_val = detect_yellow(img)
      # green_val = detect_green(img)
      # color_list = np.array([orange_val,yellow_val,green_val])
      #test_list.append(np.array([(x-min(color_list))/(max(color_list)-min(color_list))  for x in color_list])+0.01) 
      test_after_train_list7.append(avg_in_scale(crop_img))
      product_name = img_path_test_after_train7.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4

      test_after_train_list_ans7.append(product_class-1)

In [ ]:
test_weight = np.array([[-5.44277152e+04, -2.17231981e+04, -3.48126519e+04,
        -3.61506106e+04, -2.78197336e+04, -2.70166043e+04,
        -3.56884518e+04,  1.17312172e+04,  3.85330769e+04,
         4.18823862e+04,  4.70513585e+04,  4.02531196e+04,
         2.18763266e+04,  2.21466601e+04,  8.22675232e+04,
         1.25620249e+05,  1.36265908e+05,  1.15932496e+05,
         8.22201904e+04,  8.03579816e+04,  1.16228746e+05,
         8.16088602e+04,  9.39649322e+04,  9.96993704e+04,
         1.17436427e+03,  2.06685038e+04,  7.82256220e+04,
         6.50280571e+04,  1.16955118e+04, -4.13507471e+04,
        -2.97107736e+04, -1.03107034e+05, -1.08858332e+05,
        -3.92012830e+04,  2.35765694e+04, -6.66455371e+04,
        -8.22692070e+04, -6.11452251e+04, -8.68990949e+04,
        -9.38142417e+04,  4.59538107e+04,  8.88649171e+04,
         2.78682263e+04, -1.80498078e+04,  3.34517332e+04,
        -3.18669583e+04, -9.09284243e+04,  8.61000481e+03,
        -7.66910745e+04,  3.83376627e+01,  8.58207359e+04,
         5.50525947e+04,  6.07838938e+04,  5.25680064e+04,
         3.13608800e+04,  2.41247301e+04,  2.41984910e+04,
         9.26628673e+04,  1.00993997e+05,  9.80200997e+04,
         4.50624982e+04,  4.69343451e+03, -2.76277138e+04,
         3.87063672e+03, -4.67245379e+04, -1.07771951e+05,
         4.29886933e+04, -1.85764000e+04, -3.36288894e+04,
        -9.05118584e+04, -1.07854999e+03, -1.89685927e+04,
         1.38629835e+04, -5.42750242e+04, -2.81978925e+04,
        -3.73987170e+04, -7.54499733e+04, -1.09295305e+04,
        -1.52200599e+04,  1.12916350e+05,  5.67949629e+04,
         2.60374993e+04, -9.24992176e+04, -1.06659535e+05,
        -9.06145086e+03,  1.35919130e+05, -3.14628648e+04,
         7.21023361e+04, -2.66805660e+04, -1.11313450e+05,
        -1.43806848e+05,  1.41416757e+04,  6.10183886e+04,
         2.56343523e+04,  2.72212928e+04, -1.12714176e+05,
        -1.72290826e+05, -1.03543548e+05, -9.83256465e+04,
        -1.26803283e+05, -7.15824389e+04, -1.19524232e+05,
        -1.37260147e+05, -1.20692625e+05, -9.57716221e+04],
       [-2.37971784e+04,  1.93275009e+04,  6.26442718e+04,
         1.10767581e+05,  1.79504991e+05,  1.64841939e+05,
         9.71545722e+04, -8.55704844e+04, -5.41372978e+04,
        -8.12600860e+04, -1.43400018e+05, -7.90877130e+04,
        -9.21975580e+04, -1.59500196e+05,  7.49552785e+02,
        -3.35284620e+04,  6.26564104e+03, -4.72500314e+04,
        -3.55576608e+04, -1.06636081e+05, -1.25547958e+05,
        -4.32823795e+04, -8.64155133e+04, -1.93118100e+04,
        -7.67722485e+04, -8.42078427e+04, -5.89274471e+04,
        -1.28151207e+05, -6.55900900e+03, -1.89165266e+04,
         9.10708059e+04,  1.44352114e+04, -1.62274042e+04,
         2.97522532e+04, -5.59905748e+04,  6.32057828e+04,
         5.00438497e+04, -8.48388451e+04, -3.50310172e+04,
         8.18269459e+04,  9.75410682e+04,  5.93579508e+03,
         1.09118381e+04,  4.42439085e+03, -4.97351977e+04,
        -5.45696271e+04,  4.43901194e+03,  1.62853532e+05,
         1.62644502e+05, -5.08992705e+04,  4.10530599e+04,
        -6.41038977e+04, -3.15703873e+04,  4.47872452e+04,
         1.06713132e+05,  1.98422126e+04,  1.60689732e+04,
        -4.94762887e+04, -3.91026670e+04,  6.23573898e+04,
         9.07558647e+04, -1.16763691e+04, -7.59718819e+04,
         4.02874343e+03, -1.19640172e+05, -1.00261287e+05,
         1.24726538e+04,  5.70703250e+04, -9.03764300e+04,
        -5.38356222e+04,  1.07532193e+05,  5.35414991e+04,
        -1.79301381e+05, -1.49618605e+05, -6.69731672e+04,
         2.02464391e+02,  2.58362317e+04,  1.48560638e+05,
         1.29899736e+05, -7.08892869e+04, -1.14108129e+05,
        -1.03877523e+05, -9.64500929e+03,  8.91398152e+04,
         5.30560287e+04, -3.13475841e+04, -5.71535146e+04,
        -9.42331605e+04, -1.03458270e+05, -3.50164134e+04,
         9.22100943e+04,  2.20785021e+05,  1.00035966e+05,
         6.43422770e+04,  4.69624162e+04, -7.01437281e+04,
         6.61847856e+03,  8.41830797e+04,  1.31615036e+05,
         5.63695143e+04,  3.37831222e+04, -2.75677776e+04,
        -7.45535371e+04,  2.25423754e+03,  8.46548359e+04],
       [ 1.03411294e+05,  6.58868691e+04, -1.40488349e+03,
         1.74956987e+02, -1.58663653e+05, -7.61061467e+04,
        -1.03305641e+05,  1.70259571e+05,  8.45664400e+04,
         7.66895887e+04,  7.11373873e+04,  1.96505086e+04,
         1.06097823e+05,  7.74979425e+04,  7.63730223e+04,
         4.78242403e+04,  8.20988825e+03,  3.85321037e+04,
        -1.48576239e+04,  1.14385681e+05,  1.05945074e+05,
         6.12428328e+04,  7.03614005e+04, -1.02719823e+04,
         9.62911542e+04,  4.33220412e+04,  7.66809070e+04,
         1.30825525e+05,  3.50282582e+04, -9.98078817e+03,
        -1.39747364e+05, -3.98858085e+04, -2.50381932e+04,
        -4.08690567e+04,  8.92550960e+04, -4.48665804e+04,
        -2.95892886e+04,  5.17073701e+04,  1.19752765e+04,
        -1.18396162e+05, -1.21100526e+05, -3.14000615e+04,
        -9.83021509e+04, -8.67297068e+04,  2.97552038e+03,
         6.49911469e+04,  2.83609180e+04, -2.14455088e+05,
        -1.16610086e+05, -2.17243546e+04, -1.41580940e+05,
         7.58960216e+02, -5.00967071e+04, -7.92756154e+04,
        -1.52386519e+05, -7.35185665e+04, -1.19572003e+05,
        -9.24693919e+04, -2.19280661e+04,  9.82128668e+03,
         6.19098519e+03, -1.61816906e+03,  3.77994731e+03,
        -1.28182545e+05, -4.30275195e+04,  6.77064645e+04,
         2.13661686e+03, -4.81504141e+04,  1.24350223e+04,
         1.35841160e+05, -8.78489571e+04, -9.48622019e+04,
        -2.74113828e+04,  6.70673446e+04, -3.60546721e+04,
        -6.01526187e+04,  6.51873144e+04, -1.56568043e+05,
        -1.34845195e+05, -5.26014032e+04,  2.94659261e+04,
         7.32482769e+04,  1.19381162e+05,  7.34831137e+04,
        -1.55596115e+05, -1.71488382e+05,  6.18773982e+03,
        -4.22550458e+04,  3.99175778e+04,  7.08948485e+04,
         6.82557493e+04, -7.56000470e+04, -6.37159466e+04,
         1.39432843e+04, -1.09192558e+04,  1.64031678e+05,
         2.51149621e+04, -2.52267973e+04,  8.04011987e+04,
         7.74517429e+04,  1.87081762e+05,  2.22538414e+05,
         2.21677183e+05,  1.09476532e+05,  1.97931189e+04],
       [ 1.04253359e+04, -1.49415605e+04, -1.47045058e+04,
        -4.47807330e+04, -4.70459342e+04, -1.12188144e+05,
        -3.32005143e+03, -1.61692033e+04, -5.59653443e+04,
        -4.78110986e+04, -2.99391109e+04, -5.49096617e+04,
        -4.42418458e+04,  7.78645829e+04, -1.01365118e+05,
        -1.50699340e+05, -1.75800478e+05, -1.51026081e+05,
        -1.20307988e+05, -8.28298890e+04, -3.94801091e+04,
        -3.90922914e+04, -8.40467592e+04, -1.16974881e+05,
        -4.04035206e+04, -4.74757188e+04, -1.14512317e+05,
        -2.88397813e+04,  3.45597011e+02,  4.60891736e+04,
         2.66613275e+04,  7.67504318e+04,  7.01988713e+04,
         1.32391597e+04,  2.07138793e+04,  7.12683233e+04,
         7.29353603e+04,  1.09318599e+05,  8.37384672e+04,
         5.63361490e+04, -5.63475349e+04, -6.66950227e+04,
         3.47397673e+04,  6.26509492e+04,  2.47630970e+04,
         5.01035218e+04,  3.89168334e+04, -1.49847070e+04,
         2.64262305e+04,  8.11747550e+04, -1.63963786e+03,
         8.91722137e+03, -9.72826880e+03, -9.94200409e+04,
        -5.56662006e+04,  3.67542628e+04,  5.75812795e+04,
         1.32749063e+04, -2.77981037e+04, -1.37429058e+05,
        -1.19616222e+05, -3.93526514e+04,  9.75827581e+04,
         6.35653619e+04,  1.48767435e+05,  1.77968949e+05,
        -2.13216434e+04,  1.28019375e+04,  1.10449577e+05,
         1.28433050e+05, -1.36518266e+04,  5.97363655e+04,
         1.34781808e+05,  1.20628840e+05,  6.98156978e+04,
         3.95778248e+04,  4.01951721e+04,  6.99192360e+03,
         5.41022832e+03, -3.06353245e+04, -2.05083995e+04,
        -1.27798822e+04,  4.45299595e+04,  2.64294569e+04,
         3.31837831e+04, -2.05365321e+04,  7.73784109e+04,
         1.57120877e+04,  3.08140572e+04,  3.77626921e+04,
         2.17986611e+04, -1.14050795e+05, -1.01140322e+05,
        -1.01202884e+05, -1.03474936e+05, -4.53964040e+04,
        -1.10417020e+04, -5.52405810e+04, -3.95457708e+04,
        -1.72248186e+04, -6.43552271e+04, -1.68285035e+04,
         1.24702401e+03, -3.78168378e+04, -6.07436039e+04]])
#test_weight

In [ ]:

accuracy_each_epochs = []

input_nodes = 105
output_nodes = 4
bias = 1

learning_rate = 0.1


###

n = neuralNetwork(input_nodes,output_nodes,learning_rate,bias,True,test_weight)




# epochs = 2500
# for e in range(epochs):
    
#     for i in range(len(train_list)):
#         inputs =  train_list[i]        
#         targets = train_list_ans[i]
#         n.train(inputs, targets)
#         pass
#     pass

#     #test
    
#     scorecard = []

#     # go through all the records in the test data set
#     for  i in range(len(test_list)):
#         inputs =  test_list[i]
#         ans = test_list_ans[i]
      
               
#         outputs = n.query(inputs)
        
#         # the index of the highest value corresponds to the label
#         label = np.argmax(outputs)
        
#         # append correct or incorrect to list
#         if (label == ans):
            
#             scorecard.append(1)
#         else:
#             # network's answer doesn't match correct answer, add 0 to scorecard
#             scorecard.append(0)
#             pass
#         pass
#     # calculate the performance score, the fraction of correct answers
#     scorecard_array = np.asarray(scorecard)
#     #print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)
#     accuracy_each_epochs.append(scorecard_array.sum() /scorecard_array.size*100)
#     if(scorecard_array.sum() /scorecard_array.size*100 == 100):
#       break

# plt.plot(np.arange(len(accuracy_each_epochs)),np.array(accuracy_each_epochs)) 
# plt.show()
# print(max(accuracy_each_epochs))
# print(np.argmax(accuracy_each_epochs))

In [ ]:
fail_num = 1
scorecard = []
for  i in range(len(test_after_train_list)):
    inputs =  test_after_train_list[i]
    ans = test_after_train_list_ans[i]
  
            
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    
    # append correct or incorrect to list
    if (label == ans):
        
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        
        print(fail_num,' ',ans,' ',np.transpose(outputs))
        fail_num+=1
        pass
    pass
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)

In [ ]:
fail_num = 1
scorecard = []
for  i in range(len(test_after_train_list2)):
    inputs =  test_after_train_list2[i]
    ans = test_after_train_list_ans2[i]
  
            
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    
    # append correct or incorrect to list
    if (label == ans):
        
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        
        print(fail_num,' ',ans,' ',np.transpose(outputs))
        fail_num+=1
        pass
    pass
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)

In [ ]:
fail_num = 1
scorecard = []
for  i in range(len(test_after_train_list3)):
    inputs =  test_after_train_list3[i]
    ans = test_after_train_list_ans3[i]
  
            
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    
    # append correct or incorrect to list
    if (label == ans):
        
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        
        print(fail_num,' ',ans,' ',np.transpose(outputs))
        fail_num+=1
        pass
    pass
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)

In [ ]:
fail_num = 1
scorecard = []
for  i in range(len(test_after_train_list4)):
    inputs =  test_after_train_list4[i]
    ans = test_after_train_list_ans4[i]
  
            
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    
    # append correct or incorrect to list
    if (label == ans):
        
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        
        print(fail_num,' ',ans,' ',np.transpose(outputs))
        fail_num+=1
        pass
    pass
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)

In [ ]:
fail_num = 1
scorecard = []
for  i in range(len(test_after_train_list5)):
    inputs =  test_after_train_list5[i]
    ans = test_after_train_list_ans5[i]
  
            
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    
    # append correct or incorrect to list
    if (label == ans):
        
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        
        print(fail_num,' ',ans,' ',np.transpose(outputs))
        fail_num+=1
        pass
    pass
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)

In [ ]:
fail_num = 1
scorecard = []
for  i in range(len(test_after_train_list6)):
    inputs =  test_after_train_list6[i]
    ans = test_after_train_list_ans6[i]
  
            
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    
    # append correct or incorrect to list
    if (label == ans):
        
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        
        print(fail_num,' ',ans,' ',np.transpose(outputs))
        fail_num+=1
        pass
    pass
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)

In [ ]:
fail_num = 1
scorecard = []
for  i in range(len(test_after_train_list7)):
    inputs =  test_after_train_list7[i]
    ans = test_after_train_list_ans7[i]
  
            
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    
    # append correct or incorrect to list
    if (label == ans):
        
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        
        print(fail_num,' ',ans,' ',np.transpose(outputs))
        fail_num+=1
        pass
    pass
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)

In [ ]:
fail_num = 1
scorecard = []
for  i in range(len(train_list)):
    inputs =  train_list[i] 
    ans = np.argmax(train_list_ans[i])
  
            
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    
    # append correct or incorrect to list
    if (label == ans):
        
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        
        print(fail_num,' ',ans,' ',np.transpose(outputs))
        fail_num+=1
        pass
    pass
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() /scorecard_array.size*100)

In [ ]:
#train_list

In [ ]:
#train_list_ans

In [ ]:
# xdata = 7 * np.random.random(100)
# ydata = np.sin(xdata) + 0.25 * np.random.random(100)
# zdata = np.cos(xdata) + 0.25 * np.random.random(100)
x1=[]
y1=[]
z1= []
x2=[]
y2=[]
z2= []
x3=[]
y3=[]
z3= []
x4=[]
y4=[]
z4= []
for i in range(len(train_list)):
    if (np.argmax(train_list_ans[i]) == 0):
        x1.append(train_list[i][0])
        y1.append(train_list[i][1])
        z1.append(train_list[i][2])
    elif (np.argmax(train_list_ans[i]) == 1):
        x2.append(train_list[i][0])
        y2.append(train_list[i][1])
        z2.append(train_list[i][2])
    elif (np.argmax(train_list_ans[i]) == 2):
        x3.append(train_list[i][0])
        y3.append(train_list[i][1])
        z3.append(train_list[i][2])
    else:
        x4.append(train_list[i][0])
        y4.append(train_list[i][1])
        z4.append(train_list[i][2])


fig = plt.figure(figsize=(15, 15))
# Create 3D container
ax = plt.axes(projection = '3d')
# Visualize 3D scatter plot
ax.scatter3D(x1,y1,z1,color="g")
ax.scatter3D(x2,y2,z2,color="black")
ax.scatter3D(x3,y3,z3,color="red")
ax.scatter3D(x4,y4,z4,color="yellow")

ax.view_init(10, 185)
# Give labels
ax.set_xlabel('r')
ax.set_ylabel('g')
ax.set_zlabel('b')
# Save figure
#plt.savefig('3d_scatter.png', dpi = 300);

In [ ]:
n.w